In [ ]:
!pip install plotly

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import os
from plotly.subplots import make_subplots
import plotly.graph_objects as go

## **Literacy Rate By Sex (Percentage)**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Literacy Rate By Sex (Percentage)/analytical_domain.csv')
print(df.head())

                         Region       Group  Male (5+ yrs)  Female (5+ yrs)  \
0                   Koshi Urban  Analytical           89.5             74.2   
1                   Koshi Rural  Analytical           82.8             61.0   
2                 Madhesh Urban  Analytical           79.2             62.0   
3                 Madhesh Rural  Analytical           74.0             55.7   
4  Kathmandu Valley Urban (KVU)  Analytical           95.7             84.4   

   Total (5+ yrs)  Male (15+ yrs)  Female (15+ yrs)  Total (15+ yrs)  
0            81.2            87.6              69.4             77.6  
1            72.5            79.0              55.8             66.8  
2            70.2            75.6              52.9             61.2  
3            62.9            65.1              52.9             52.6  
4            89.6            94.2              82.3             88.2  


In [ ]:
fig = px.bar(df, x='Region', y=['Male (5+ yrs)', 'Female (5+ yrs)', 'Total (5+ yrs)'],
             title="Literacy Rate by Region (5+ yrs)", barmode='group',
             labels={'value': 'Literacy Rate (%)', 'variable': 'Category'})
new_names = {
    'Male (5+ yrs)': 'Male',
    'Female (5+ yrs)': 'Female',
    'Total (5+ yrs)': 'Total'
}
fig.for_each_trace(lambda t: t.update(name = new_names[t.name]))

fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Literacy Rate By Sex (Percentage)/consumption_quintile.csv')
df.columns = df.columns.str.strip()

df_5plus = df.melt(id_vars=['Quintile'],
                   value_vars=['Male (5+ yrs)', 'Female (5+ yrs)', 'Total (5+ yrs)'],
                   var_name='Gender', value_name='Literacy Rate')
df_5plus['Age Group'] = '5+ yrs'
df_15plus = df.melt(id_vars=['Quintile'],
                    value_vars=['Male (15+ yrs)', 'Female (15+ yrs)', 'Total (15+ yrs)'],
                    var_name='Gender', value_name='Literacy Rate')
df_15plus['Age Group'] = '15+ yrs'
df_long = pd.concat([df_5plus, df_15plus])
df_long['Gender'] = df_long['Gender'].str.replace(r'\s*\(\d+\+\s*yrs\)', '', regex=True)
fig = px.line(df_long,
              x='Quintile', y='Literacy Rate', color='Gender',
              facet_col='Age Group',
              markers=True, title='Literacy Rate by Consumption Quintile (Advanced View)')
fig.show()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Literacy Rate By Sex (Percentage)/poverty_status.csv')
df.columns = df.columns.str.strip()

# Prepare heatmap matrix and labels
z = df.drop('Status', axis=1).values
x = df.columns[1:]
y = df['Status']

# Create annotated heatmap
fig = go.Figure(data=go.Heatmap(
    z=z,
    x=x,
    y=y,
    colorscale='Turbo',
    colorbar=dict(title="Literacy Rate (%)"),
    hoverongaps=False,
    hovertemplate='Status: %{y}<br>Category: %{x}<br>Literacy Rate: %{z:.1f}%',
))

# Add annotations (text values) inside heatmap cells
for yi, row in enumerate(z):
    for xi, val in enumerate(row):
        fig.add_annotation(
            x=x[xi],
            y=y[yi],
            text=f'{val:.1f}%',
            showarrow=False,
            font=dict(color='black', size=14),
            yshift=0
        )

# Layout improvements
fig.update_layout(
    title='Literacy Rate by Poverty Status and Category',
    xaxis_title='Category',
    yaxis_title='Poverty Status',
    yaxis=dict(autorange='reversed'),
    template='plotly_dark',
    width=900,
    height=500,
    margin=dict(l=100, r=100, t=100, b=100),
    font=dict(family='Helvetica', size=16)
)

fig.show()



In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Literacy Rate By Sex (Percentage)/province.csv')
df.columns = df.columns.str.strip()
df = df.sort_values(by='Total (5+ yrs)', ascending=True)

fig = go.Figure()

# Male bars
fig.add_trace(go.Bar(
    y=df['Region'],
    x=df['Male (5+ yrs)'],
    name='Male (5+ yrs)',
    orientation='h',
    marker=dict(color='royalblue'),
    hovertemplate='Male (5+ yrs): %{x:.1f}%<extra></extra>'
))

# Female bars
fig.add_trace(go.Bar(
    y=df['Region'],
    x=df['Female (5+ yrs)'],
    name='Female (5+ yrs)',
    orientation='h',
    marker=dict(color='lightpink'),
    hovertemplate='Female (5+ yrs): %{x:.1f}%<extra></extra>'
))

# Add total literacy as a line scatter for contrast
fig.add_trace(go.Scatter(
    y=df['Region'],
    x=df['Total (5+ yrs)'],
    mode='markers+lines',
    name='Total (5+ yrs)',
    marker=dict(color='mediumseagreen', size=12, symbol='circle'),
    line=dict(color='mediumseagreen', width=2),
    hovertemplate='Total (5+ yrs): %{x:.1f}%<extra></extra>'
))

fig.update_layout(
    title='Literacy Rates (5+ yrs) by Province and Sex',
    barmode='group',
    xaxis=dict(title='Literacy Rate (%)', range=[0, 100]),
    yaxis=dict(title='Province'),
    template='plotly_white',
    width=900,
    height=600,
    legend=dict(title='Category'),
    margin=dict(l=120, r=50, t=80, b=50),
    font=dict(size=14)
)

fig.show()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Literacy Rate By Sex (Percentage)/urban_rural.csv')
df.columns = df.columns.str.strip()
df = df.sort_values(by='Total (5+ yrs)', ascending=False)

fig = go.Figure()

# Add Male (5+ yrs) bar
fig.add_trace(go.Bar(
    x=df['Region'],
    y=df['Male (5+ yrs)'],
    name='Male (5+ yrs)',
    marker=dict(color='royalblue'),
    hovertemplate='Male (5+ yrs): %{y:.1f}%<extra></extra>'
))

# Add Female (5+ yrs) bar
fig.add_trace(go.Bar(
    x=df['Region'],
    y=df['Female (5+ yrs)'],
    name='Female (5+ yrs)',
    marker=dict(color='lightpink'),
    hovertemplate='Female (5+ yrs): %{y:.1f}%<extra></extra>'
))

# Add Total (5+ yrs) as a line
fig.add_trace(go.Scatter(
    x=df['Region'],
    y=df['Total (5+ yrs)'],
    name='Total (5+ yrs)',
    mode='markers+lines',
    marker=dict(color='mediumseagreen', size=12),
    line=dict(color='mediumseagreen', width=3),
    hovertemplate='Total (5+ yrs): %{y:.1f}%<extra></extra>'
))

# Layout customization
fig.update_layout(
    title='Literacy Rate (5+ yrs) by Urban-Rural Regions',
    xaxis_title='Region',
    yaxis_title='Literacy Rate (%)',
    yaxis=dict(range=[0, 100]),
    barmode='group',
    template='plotly_white',
    width=900,
    height=600,
    legend=dict(title='Category'),
    margin=dict(l=80, r=50, t=80, b=80),
    font=dict(size=14)
)

fig.show()

## **Net Enrollement Rate by Level Of Education & Sex**

In [ ]:
import pandas as pd
import plotly.subplots as sp
import plotly.graph_objects as go

# Load datasets from file paths
basic_df = pd.read_csv('/content/drive/MyDrive/Net Enrollement Rate by Level Of Education & Sex/net_enrollment_basic_level.csv')
secondary_df = pd.read_csv('/content/drive/MyDrive/Net Enrollement Rate by Level Of Education & Sex/net_enrollment_secondary_level.csv')
tertiary_df = pd.read_csv('/content/drive/MyDrive/Net Enrollement Rate by Level Of Education & Sex/net_enrollment_tertiary_level.csv')

# Strip any extra spaces
basic_df.columns = basic_df.columns.str.strip()
secondary_df.columns = secondary_df.columns.str.strip()
tertiary_df.columns = tertiary_df.columns.str.strip()

# Rename tertiary columns to match others for consistency
tertiary_df.rename(columns={'Boys': 'Male', 'Girls': 'Female'}, inplace=True)

# Create radar chart subplots
fig = sp.make_subplots(
    rows=1, cols=3,
    specs=[[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]],
    subplot_titles=["Basic Level", "Secondary Level", "Tertiary Level"]
)

# Basic Level Radar
fig.add_trace(go.Scatterpolar(
    r=basic_df['Male'], theta=basic_df['Province'],
    name='Male (Basic)', marker_color='royalblue'), row=1, col=1)
fig.add_trace(go.Scatterpolar(
    r=basic_df['Female'], theta=basic_df['Province'],
    name='Female (Basic)', marker_color='lightpink'), row=1, col=1)
fig.add_trace(go.Scatterpolar(
    r=basic_df['Total'], theta=basic_df['Province'],
    name='Total (Basic)', marker_color='mediumseagreen'), row=1, col=1)

# Secondary Level Radar
fig.add_trace(go.Scatterpolar(
    r=secondary_df['Male'], theta=secondary_df['Province'],
    name='Male (Secondary)', marker_color='royalblue', showlegend=False), row=1, col=2)
fig.add_trace(go.Scatterpolar(
    r=secondary_df['Female'], theta=secondary_df['Province'],
    name='Female (Secondary)', marker_color='lightpink', showlegend=False), row=1, col=2)
fig.add_trace(go.Scatterpolar(
    r=secondary_df['Total'], theta=secondary_df['Province'],
    name='Total (Secondary)', marker_color='mediumseagreen', showlegend=False), row=1, col=2)

# Tertiary Level Radar
fig.add_trace(go.Scatterpolar(
    r=tertiary_df['Male'], theta=tertiary_df['Province'],
    name='Male (Tertiary)', marker_color='royalblue', showlegend=False), row=1, col=3)
fig.add_trace(go.Scatterpolar(
    r=tertiary_df['Female'], theta=tertiary_df['Province'],
    name='Female (Tertiary)', marker_color='lightpink', showlegend=False), row=1, col=3)
fig.add_trace(go.Scatterpolar(
    r=tertiary_df['Total'], theta=tertiary_df['Province'],
    name='Total (Tertiary)', marker_color='mediumseagreen', showlegend=False), row=1, col=3)

# Layout
fig.update_layout(
    title_text="Net Enrollment Rates by Province, Education Level & Sex",
    height=700,
    width=1600,
    template='plotly_white',
    polar=dict(radialaxis=dict(range=[0, 105]))
)

fig.show()
